# Relatório e código do Trabalho Prático de Análise e Transformação de Dados

## 1. Importar Sinais Necessários

Para importar os dados necessários, começamos por criar uma função que lê os ficheiros presentes numa pasta com os DataSets a analisar, e verfica se o formato do nome dos mesmos está de acordo com o necessário

Começaremos a nossa análise por ler os ficheiros, fazendo as verificações básicas á possibilidade de execução

In [1]:
%Taxa de amostragem dos dados
fs = 50;
if ~isfile("data.mat")
    data = lerDados();
    data = lerLabel(data);
    activity_label = lerActivityLabel();
    if(~isa(data,'cell'))
        return
    end
end

In [ ]:
%%file lerDados.m
function data = lerDados()
PATH = "Data_PL2";
ficheiro_path = dir(PATH);
numero_ficheiros = length(ficheiro_path)-2;
data = cell(numero_ficheiros,1);
string_patern = digitsPattern;

for i = 1:numero_ficheiros
    dados_exp = str2double(extract(ficheiro_path(i+2).name, string_patern));
    if(length(dados_exp) ~= 2)
        warning("Erro a ler conteudo da pasta, verifique a sua pasta e o parsing do ficheiro " + ficheiro_path(i+2).name);
        data = [];
        break
    end
    data{i,1}.data = (load(PATH + '\' + ficheiro_path(i+2).name));
    data{i,1}.user = dados_exp(2); %Guardar utilizador
    data{i,1}.exp_num = dados_exp(1); %Guardar numero experiencia
end

In [ ]:
%%file lerLabel.m

function data = lerLabel(data)
PATH = "Help\labels.txt";
temp = load(PATH);
%temp = temp(ismember(temp(:, 1),experiencias), :);
for i = 1:length(data)
    exp_labels = temp(:,1) == data{i}.exp_num;
    data{i}.activity.number = num2cell(temp(exp_labels, 3));
    data{i}.activity.start = num2cell(temp(exp_labels, 4));
    data{i}.activity.end = num2cell(temp(exp_labels, 5));
end

In [ ]:
%%file lerActivityLabel.m

function label = lerActivityLabel()
PATH = "Help\activity_labels.txt";

temp = table2cell(readtable(PATH));
label = temp(:, 2);

## 2. Representar graficamente os sinais importados, identificando a atividade a que cada fragmento corresponde.

In [ ]:
%plot -b inline -w 1500 -h 1500
experiencias = zeros(length(data),1);
for i = 1:length(data)
    experiencias(i) = data{i}.exp_num;
end
[indx,~] = listdlg('ListString', num2str(experiencias), 'SelectionMode','single', 'Name', ...
            'Escolha uma experiencia para visualizar');
%user_plot(data, activity_label, indx, fs);

In [ ]:
%%file user_plot.m
function user_plot(data, activity_label, index, fs)

colors = ['#4DBEEE';'#D95319';'#EDB120';'#7E2F8E';'#77AC30';'#A2142F';'#0072BD';'#00FF00';'#FF0000'; ...
          '#0000FF';'#00FFFF';'#FF00FF'];

figure(1);
indexes = 1:length(data{index}.data);
indexes = indexes./fs./60;

%Adicionar dados no plot(LETRA X)
letter = 88;
for ax = 1:3
    subplot(3,1,ax);
    hold on
    start = 1;
    for i = 1: length(data{index}.activity.number)
        label_begin = cell2mat(data{index}.activity.start(i));
        label_end = cell2mat(data{index}.activity.end(i));
        if label_begin > start
            data_range = start:label_begin;
            data_output = data{index}.data(data_range,ax);
            plot(indexes(data_range),data_output, 'k');
        end
        %obter atividade
        actv_num = cell2mat(data{index}.activity.number(i));
        data_range = label_begin:label_end;
        data_output = data{index}.data(data_range,ax);
        plot(indexes(data_range),data_output, 'color', colors(actv_num,:));
        start = label_end;
        text_pos_y = max(data_output) + 0.2;
        txt = activity_label(actv_num);
        t = text(double(indexes(round((label_begin+label_end)/2))), text_pos_y,txt , 'Interpreter' , 'none');
        t.FontSize = 12;
        t.Rotation = 90;
    end
    data_range = (start:length(data{index}.data(:,ax)));
    data_output = data{index}.data(data_range,ax);
    plot(indexes(data_range), data_output, 'k');
    xlabel("Time (min)", 'FontSize' , 15);
    ylabel(strcat("AAC ", char(letter)), 'FontSize' , 15);
    letter = letter + 1;
    hold off
end

## 3. Calcular DFT associado a cada atividade, para todos os utilizadores 

## Análise teórica das janelas

### Uma pequena Introdução
Numa primeira fase, precisamos de entender os diversos fenómenos e consequencias dos mesmos na nossa análise, para isso precisamos de perceber o funcionamento da transformada de Fourier, no nosso caso, em tempo discreto (Devido ás limitações computacionais).

Ao aplicar esta transformada pretendemos obter uma **representação em frequencia do sinal original**, para isso, vamos decompomo-lo numa soma cossenos, obtendo assim as frequencias do sinal original, bem como a sua amplitude **num determinado periodo**.

![](Imagens\DFT.png)

Sendo 
- m&Omega;<sub>0</sub> - as frequências angulares
- &Theta;m - as fases das sinusóides


### O efeito das janelas

Iniciando a análise de janelas numa perspetiva teórica, sabemos que o sucesso ou não da análise de sinais com uso da transformada DTF pode depender, em grande escala, do tipo de janela a utilizar.


- Spectral leakage: Para entende-lo, temos de relembrar que a transformada DFT considera em repetição periodica do mesmo sinal, isto causa
- Noise Bandwidth

Vamos apenas realizar uma breve enunciação de pontos fracos e fortes de cada  

---
Neste ponto começamos a análise de dados com aplicação da DFT a todos os fragmento referente a uma atividade, para todos utilizadores.
Para melhorar a nossa análise posterior, guardámos também as larguras de erro dos sinais, para cada largura aplicada, isto porque pretendemos escolher entre quatro janelas, a que achamos ser a mais vantajosa para o projeto em questão, tendo em conta as análises de frequencia, para cada atividade






In [2]:
if ~isfile("data.mat")
    data = DFT(data, fs);

else
    disp("Loading already existing file");
end


In [27]:
%Mostrar graficos erro das janelas por atividade
mean_size = get_mean_sizes(data);

[act_in,~] = listdlg('ListString', activity_label, 'Name','Escolha uma experiencia para visualizar');

mean_activity_size = mean_size(act_in);

temp_window = hamming(mean_activity_size);
hamm = wvtool(temp_window);
hamm.Name = 'Janela de Hamming';
disp(strcat("Mean Hamming Window Noise Bandwith to activity ", activity_label(act_in)));
disp(enbw(hamming_window, fs));

temp_window = hann(mean_activity_size);
h = wvtool(temp_window);
h.Name = 'Janela de Hann';
disp(strcat("Mean Hann Window Noise Bandwith to activity ", activity_label(act_in)));
disp(enbw(temp_window, fs));


temp_window = flattopwin( mean_size(act_in));
flattop = wvtool(temp_window);
flattop.Name = 'Janela Flatop';
disp(strcat("Mean Flat Top Window Noise Bandwith to activity ", activity_label(act_in)));
disp(enbw(temp_window, fs));


%Make a window viewer with all windows for visual comparision
all_win = wvtool(hamming(mean_activity_size), hann(mean_activity_size), flattopwin( mean_size(act_in)));
all_win.Name = 'Juncão de todas as janelas';

Mean Hamming Window Noise Bandwith to activity WALKING_UPSTAIRS
    0.0727

Mean Hann Window Noise Bandwith to activity WALKING_UPSTAIRS
    0.1236

Mean Flat Top Window Noise Bandwith to activity WALKING_UPSTAIRS
    0.3106




## Análise das janelas, por atividade

|Tipo de atividade| Atividade      |                               | Hamming     | Hann     | Flat Top |
|:--- | :---  | :---  | :---: | :---:   |:---: |
|**Dinamicas:**|
|                 | Walking        |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 |WALKING_UPSTRS|                              |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | WALKING_DWSTRS|                            |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          | 0.0727      |   0.1236 |   0.3106 |
|**Estáticas:**   |
|                 | SITTING        |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | STANDING       |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | LAYING         |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|**De transição:**|
|                 | STAND_TO_SIT   |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | SIT_TO_STAND   |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | SIT_TO_LIE     |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | LIE_TO_SIT     |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | STAND_TO_LIE   |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | LIE_TO_STAND   |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |
|                 | STANDING       |                               |             |          |          |
|                 |                | Leakege Factor                |             |          |          |
|                 |                | Relative Side Lobe Atenuation |             |          |          |
|                 |                | Mainlobe width (-3db)         |             |          |          |
|                 |                | Mean Noise Bandwidth          |             |          |          |

In [25]:
%%file get_mean_sizes.m
function mean_size = get_mean_sizes(data)
    exp_indexes = 1:length(data);
    mean_size = zeros(12, 1);
    for i = 1:12
        n_am = 0;
        n_sum = 0;
        for experience = exp_indexes
            indexes = find(i == cell2mat(data{experience}.activity.number));
            len = cell2mat(data{experience}.activity.end(indexes)) - cell2mat(data{experience}.activity.start(indexes));
            n_sum = n_sum + sum(len);
            n_am = n_am + length(len);
        end
        mean_size(i) = round(n_sum / n_am);
    end
end

Created file 'C:\Users\User\Desktop\2ºAno\ATD\PROJETO\get_mean_sizes.m'.


In [26]:
%plot -b inline -w 1500 -h 700

window_names = {'RETANGULAR', 'HAMMING', 'HANN', 'FLAT TOP'};

[exp_in,~] = listdlg('ListString', num2str(experiencias), 'SelectionMode','single', 'Name','Escolha uma experiencia para visualizar');
[act_in,~] = listdlg('ListString', activity_label, 'Name','Escolha uma experiencia para visualizar');
[windows,~] = listdlg('ListString', window_names, 'Name','Escolha uma ou mais janelas para visualizar');


data_act_index = [];
for i = 1:length(act_in)
    indexes = find(cell2mat(data{exp_in}.activity.number) == act_in(i))';
    data_act_index = [data_act_index indexes];
end

DFT_PLOTS(data, exp_in, data_act_index, fs, activity_label, windows)


Unrecognized function or variable 'experiencias'.



Error: -1

In [16]:
%%file DFT.m

function data = DFT(data)
for experience = 1:length(data)
    n_act_users = length(data{experience}.activity.number);
    for user_activity = 1:n_act_users
        
        start_index = cell2mat(data{experience}.activity.start(user_activity,1));
        end_index = cell2mat(data{experience}.activity.end(user_activity,1));
        data_range = start_index:end_index;
        data_s = length(data_range);
        for ax = 1:3
            %DFT Janela Retangular
            data_to_DFT = data{experience}.data(data_range, ax);
            data{experience}.activity.DFT_RECT{user_activity, ax} =  fftshift(fft(data_to_DFT));

            %DFT Janela Hamming
            window = hamming(data_s);
            data_to_DFT = data{experience}.data(data_range, ax) .* window;
            data{experience}.activity.DFT_HAMMING{user_activity, ax} =  fftshift(fft(data_to_DFT));
         
            %DFT Janela Hann
            window = hann(data_s);
            data_to_DFT = data{experience}.data(data_range, ax) .* window;
            data{experience}.activity.DFT_HANN{user_activity, ax} =  fftshift(fft(data_to_DFT));

            %DFT Janela Flat top
            window = flattopwin(data_s);
            data_to_DFT = data{experience}.data(data_range, ax) .* window;
            data{experience}.activity.DFT_FLATOP{user_activity, ax} =  fftshift(fft(data_to_DFT));
        end
    end
end

Created file 'C:\Users\User\Desktop\2ºAno\ATD\PROJETO\DFT.m'.


In [ ]:
%%file DFT_PLOTS.m

function DFT_PLOTS(data, experience, data_act_index, Fs, activity_label, windows)
colors = ['#00FF00';'#FF0000';'#0000FF';'#00FFFF';'#FF00FF'];


for user_activity_index = 1:length(data_act_index)
   figure
   index = 0;
   user_activity = data_act_index(user_activity_index);
   start_index = cell2mat(data{experience}.activity.start(user_activity,1));
   end_index = cell2mat(data{experience}.activity.end(user_activity,1));
   data_range = start_index:end_index;
   N = length(data_range);
   if mod(N,2) == 0
            f = -Fs/2:Fs/N:Fs/2-Fs/N;
   else
            f = -Fs/2 + Fs/N/2:Fs/N:Fs/2-Fs/N/2;
   end


   actv_num = cell2mat(data{experience}.activity.number(user_activity));
   ylabel(activity_label(actv_num));
   
   for ax = 1:3
        color = 1;
        subplot(1, 3, index + ax);
        hold on
        if ismember(1, windows)
            DFT_RECT = data{experience}.activity.DFT_RECT{user_activity, ax};
            plot(f, abs(DFT_RECT), '-', 'color', colors(color, :));
        end
        color = color + 1;
        if ismember(2, windows)
            DFT_HANN = data{experience}.activity.DFT_HANN{user_activity, ax};
            plot(f, abs(DFT_HANN), '-', 'color', colors(color, :));
        end
        color = color + 1;
        if ismember(3, windows)
            DFT_HAMMING = data{experience}.activity.DFT_HAMMING{user_activity, ax};
            plot(f, abs(DFT_HAMMING), '-', 'color', colors(color, :));
        end
        color = color + 1;
        if ismember(4, windows)
            DFT_FLATOP = data{experience}.activity.DFT_FLATOP{user_activity, ax};
            plot(f, abs(DFT_FLATOP), '-', 'color', colors(color, :));
        end
        if ax == 1
            xlabel("X");
            ylabel(activity_label(actv_num), 'Interpreter' , 'none');
        elseif ax == 2
            xlabel("Y");
        else
            xlabel("Z");
        end
        
   end
   
   index = index + 3;
end